---
title: "Analyse E-commerce avec Polars"
author: "Gaël Penessot"
date: today
date-format: "DD/MM/YYYY"
format: 
  html:
    code-fold: true
    theme: cosmo
    toc: true
    toc-depth: 3
    fig-width: 8
    fig-height: 6
execute:
  echo: false
  warning: false
filters:
   - quarto
---

In [ ]:
#| label: setup
#| include: false
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
import json
import os

# Chargement des résultats
results_dir = "/home/gael/Documents/Github/polars-ecommerce-analysis/data/processed/report_20250119_003031/results"
def load_results(filename):
    filepath = os.path.join(results_dir, filename)
    if filepath.endswith('.json'):
        with open(filepath, 'r') as f:
            return json.load(f)
    return pl.read_csv(filepath)

# Chargement des données
global_kpis = load_results('global_kpis.json')
customer_metrics = load_results('customer_metrics.csv')
price_analysis = load_results('price_analysis.csv')
top_products = load_results('top_products.csv')
temporal_analysis = {
    k: load_results(f'temporal_{k}.csv') 
    for k in ['daily', 'weekday', 'hourly']
}

# Résumé Exécutif

Cette analyse présente une vue d'ensemble de notre activité e-commerce basée sur les données disponibles. Les points clés incluent :

- Chiffre d'affaires total : €{{format(global_kpis['total_revenue'], ',.2f')}}
- Nombre total de commandes : {{format(global_kpis['total_orders'], ',d')}}
- Panier moyen : €{{format(global_kpis['average_order_value'], ',.2f')}}

## Points Saillants

1. Performance Générale
   - {{global_kpis['total_customers']}} clients actifs
   - {{global_kpis['total_products']}} produits différents vendus
   - Moyenne de {{format(global_kpis['average_items_per_order'], '.1f')}} articles par commande

2. Répartition des Ventes

In [ ]:
#| label: fig-sales-distribution
#| fig-cap: Distribution des ventes par catégorie de prix

fig = px.bar(price_analysis, 
             x='PriceCategory', 
             y='TotalRevenue',
             title="Répartition du CA par catégorie de prix",
             labels={'PriceCategory': 'Catégorie de prix', 
                    'TotalRevenue': 'Chiffre d\'affaires (€)'})
fig.show()

# Analyse Détaillée

## 1. Analyse Temporelle

### 1.1 Évolution des Ventes

In [ ]:
#| label: fig-sales-evolution
#| fig-cap: Évolution quotidienne des ventes

fig = px.line(temporal_analysis['daily'], 
              x='OrderDate', 
              y='Revenue',
              title="Évolution du chiffre d'affaires quotidien",
              labels={'Revenue': 'CA (€)', 'OrderDate': 'Date'})
fig.show()

### 1.2 Distribution Hebdomadaire

In [ ]:
#| label: fig-weekly-pattern
#| fig-cap: Distribution des ventes par jour de la semaine

fig = px.bar(temporal_analysis['weekday'],
             x='WeekDay',
             y='Revenue',
             title="CA par jour de la semaine",
             labels={'WeekDay': 'Jour', 'Revenue': 'CA (€)'})
fig.show()

## 2. Analyse des Produits

### 2.1 Top 10 des Produits

In [ ]:
#| label: fig-top-products
#| fig-cap: Top 10 des produits par chiffre d'affaires

top_10 = top_products.head(10)
fig = px.bar(top_10,
             x='Description',
             y='TotalRevenue',
             title="Top 10 des produits",
             labels={'Description': 'Produit', 
                    'TotalRevenue': 'CA total (€)'})
fig.update_layout(xaxis_tickangle=45)
fig.show()

### 2.2 Analyse des Prix
Cette section examine la distribution des prix et leur impact sur les ventes.


In [ ]:
#| label: fig-price-analysis
#| fig-cap: Analyse des catégories de prix

fig = go.Figure()
fig.add_trace(go.Bar(
    name="Nombre de produits",
    x=price_analysis['PriceCategory'],
    y=price_analysis['NumberProducts']
))
fig.add_trace(go.Bar(
    name="Nombre de commandes",
    x=price_analysis['PriceCategory'],
    y=price_analysis['NumberOrders']
))
fig.update_layout(barmode='group',
                 title="Comparaison des métriques par catégorie de prix")
fig.show()

## 3. Analyse des Clients

### 3.1 Segmentation RFM

In [ ]:
#| label: fig-rfm-segments
#| fig-cap: Répartition des segments RFM

segment_counts = (customer_metrics
    .group_by('RFM_Segment')
    .agg(CustomerID=pl.count('CustomerID'))
    .sort('CustomerID', reversed=True))

fig = px.pie(segment_counts,
             names='RFM_Segment',
             values='CustomerID',
             title="Répartition des segments clients")
fig.show()

### 3.2 Valeur des Segments

In [ ]:
#| label: fig-segment-value
#| fig-cap: Valeur moyenne par segment

segment_value = (customer_metrics
    .group_by('RFM_Segment')
    .agg(MonetaryValue=pl.mean('MonetaryValue'))
    .sort('MonetaryValue', reversed=True))

fig = px.bar(segment_value,
             x='RFM_Segment',
             y='MonetaryValue',
             title="Valeur moyenne des clients par segment",
             labels={'RFM_Segment': 'Segment',
                    'MonetaryValue': 'Valeur moyenne (€)'})
fig.show()

# Recommandations Stratégiques

1. **Optimisation de la gamme de prix**
   - Les produits {{price_analysis[0]['PriceCategory']}} génèrent le plus de revenus
   - Opportunité d'expansion dans la catégorie {{price_analysis[-1]['PriceCategory']}}

2. **Gestion de la clientèle**
   - Focus sur la rétention des "Champions" ({{segment_counts[0]['CustomerID']}} clients)
   - Programme de réactivation pour les clients dormants

3. **Optimisation temporelle**
   - Renforcer les ventes pendant les pics identifiés
   - Développer des actions pour les périodes creuses

# Conclusion et Prochaines Étapes

Cette analyse met en évidence plusieurs opportunités d'amélioration :

1. **Court terme**
   - Optimisation du stock des top 10 produits
   - Campagnes ciblées par segment RFM

2. **Moyen terme**
   - Développement de la gamme de prix moyenne
   - Programme de fidélisation basé sur les insights RFM

3. **Long terme**
   - Expansion de la gamme premium
   - Automatisation du suivi client